<a href="https://colab.research.google.com/github/aswanthoffl/Video-Caption-Generation/blob/main/model/Video_Caption_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1082e728-273c-0836-9576-8bdef6e77c55)


In [3]:
#@title Setup
import os, subprocess


def setup():
    install_cmds = [
        ['pip', 'install', 'gradio'],
        ['pip', 'install', 'open_clip_torch'],
        ['pip', 'install', 'clip-interrogator'],
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     

In [4]:
caption_model_name = 'blip-large'
clip_model_name = 'ViT-L-14/openai'

import gradio as gr
from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = clip_model_name
config.caption_model_name = caption_model_name
ci = Interrogator(config)

Loading caption model blip-large...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Loading CLIP model ViT-L-14/openai...


100%|███████████████████████████████████████| 933M/933M [00:09<00:00, 95.5MiB/s]
ViT-L-14_openai_artists.safetensors: 100%|██████████| 16.2M/16.2M [00:00<00:00, 123MB/s]
ViT-L-14_openai_flavors.safetensors: 100%|██████████| 155M/155M [00:00<00:00, 212MB/s]
ViT-L-14_openai_mediums.safetensors: 100%|██████████| 146k/146k [00:00<00:00, 26.2MB/s]
ViT-L-14_openai_movements.safetensors: 100%|██████████| 307k/307k [00:00<00:00, 41.7MB/s]
ViT-L-14_openai_trendings.safetensors: 100%|██████████| 111k/111k [00:00<00:00, 39.5MB/s]
ViT-L-14_openai_negative.safetensors: 100%|██████████| 63.2k/63.2k [00:00<00:00, 36.6MB/s]

Loaded CLIP model and data in 23.88 seconds.


**Image To Prompt**

In [ ]:
'''
def image_to_prompt(image, mode):
    ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    image = image.convert('RGB')
    if mode == 'best':
        return ci.interrogate(image)
    elif mode == 'classic':
        return ci.interrogate_classic(image)
    elif mode == 'fast':
        return ci.interrogate_fast(image)
    elif mode == 'negative':
        return ci.interrogate_negative(image)

def prompt_tab():
    with gr.Col
        with gr.Row():
            image = gr.Image(type='pil', label="Image")
            with gr.Column():
                mode = gr.Radio(['best', 'fast', 'classic', 'negative'], label='Mode', value='best')
        prompt = gr.Textbox(label="Prompt")
    button = gr.Button("Generate prompt")
    button.click(image_to_prompt, inputs=[image, mode], outputs=prompt)

with gr.Blocks() as ui:
    with gr.Tab("Prompt"):
        prompt_tab()

ui.launch(show_api=False, debug=False)

'''

**Video To Prompt**

In [6]:
import cv2
import re
from PIL import Image

def image_to_prompt(image, mode):
    ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    image = image.convert('RGB')
    if mode == 'best':
        return ci.interrogate(image)
    elif mode == 'classic':
        return ci.interrogate_classic(image)
    elif mode == 'fast':
        return ci.interrogate_fast(image)
    elif mode == 'negative':
        return ci.interrogate_negative(image)


# Function to extract one frame per second from a video
def extract_frames_one_per_second(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    frames_to_extract = int(frame_rate)  # Extract one frame per second

    for i in range(0, total_frames, int(frame_rate)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames


# Function to print the first sentence of a prompt

def print_first_sentences(prompt):
    sentences = re.split(r',', prompt)

    # Print the first three sentences
    for i in range(min(1, len(sentences))):
        print(sentences[i])

# Process frames using image_to_prompt function
def process_video_frames(video_path, mode):
    frames = extract_frames_one_per_second(video_path)
    prompts = []

    for frame in frames:
        pil_image = Image.fromarray(frame)
        prompt = image_to_prompt(pil_image, mode)
        print_first_sentences(prompt)
        prompts.append(prompt)

    return prompts

# Usage example
video_path = '/content/drive/MyDrive/Colab Notebooks/Matplotlib/Rainy Day [short 30 sec animation].mp4'
mode = 'best'  # Change mode as needed
prompts = process_video_frames(video_path, mode)

100%|██████████| 50/50 [00:00<00:00, 208.57it/s]


cartoon of a woman sitting in a room with a laptop


100%|██████████| 50/50 [00:00<00:00, 212.33it/s]


cartoon of a woman sitting in a room with a laptop


100%|██████████| 50/50 [00:00<00:00, 210.01it/s]


...

there is a man that is standing in the dark with a cell phone


100%|██████████| 50/50 [00:00<00:00, 137.82it/s]


there is a man that is standing in the dark with a cell phone


100%|██████████| 50/50 [00:00<00:00, 215.95it/s]

there is a man that is standing in the dark with a cell phone


In [7]:
# Assuming prompts is a list of strings containing generated prompts for each frame
new_list=[]

# Function to print the first sentence of a prompt
def print_first_sentences(prompts):
    for prompt in prompts:
        sentences = prompt.split(',')
        if sentences:
            print(sentences[0])
            new_list.append(sentences[0])

# Usage example
print_first_sentences(prompts)

cartoon of a woman sitting in a room with a laptop
cartoon of a woman sitting in a room with a laptop
cartoon of a woman sitting in a room with a laptop
cartoon of a person sitting in a room with a window and a cat
cartoon of a woman sitting in a room with a laptop
cartoon of a person standing in a room with a teddy bear
there is a cartoon of a person standing in a room with a teddy bear
cartoon dog sitting in a window box in the rain
cartoon dog sitting in a window box in the rain
cartoon of a window with a rain falling down on a planter
cartoon of a woman in a bed with a cat and a dog
cartoon of a cat and a cat laying in a bed
there is a cat that is laying on a bed in a room
there is a drawing of a woman holding a green jacket
cartoon of a person with a hat on and a green shirt
cartoon of a man with a hat and a microphone
cartoon of a person wearing green socks and blue socks
a close up of a person's legs in blue boots
a close up of a person's feet in a pair of blue boots
cartoon of 

In [8]:
new_list

['cartoon of a woman sitting in a room with a laptop',
 'cartoon of a woman sitting in a room with a laptop',
 'cartoon of a woman sitting in a room with a laptop',
 'cartoon of a person sitting in a room with a window and a cat',
 'cartoon of a woman sitting in a room with a laptop',
 'cartoon of a person standing in a room with a teddy bear',
 'there is a cartoon of a person standing in a room with a teddy bear',
 'cartoon dog sitting in a window box in the rain',
 'cartoon dog sitting in a window box in the rain',
 'cartoon of a window with a rain falling down on a planter',
 'cartoon of a woman in a bed with a cat and a dog',
 'cartoon of a cat and a cat laying in a bed',
 'there is a cat that is laying on a bed in a room',
 'there is a drawing of a woman holding a green jacket',
 'cartoon of a person with a hat on and a green shirt',
 'cartoon of a man with a hat and a microphone',
 'cartoon of a person wearing green socks and blue socks',
 "a close up of a person's legs in blue b